# VerbaFlow - Experimentação com Agentes

Este notebook demonstra o uso dos agentes CrewAI do VerbaFlow para classificação e enriquecimento de textos.

## Objetivo

Testar o sistema multi-agente sem a interface Streamlit, permitindo experimentação direta com os componentes.


In [ ]:
# Importações necessárias
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Adicionar diretório raiz ao path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Importar módulos do VerbaFlow
from src.utils import fetch_newsgroups_samples, clean_text, extract_ground_truth_from_filename, get_text_from_file
from src.agents import get_llm, create_analyst_agent, create_researcher_agent, create_editor_agent
from src.tasks import create_classification_task, create_enrichment_task, create_reporting_task
from crewai import Crew, Process

print("✅ Módulos importados com sucesso!")


✅ Módulos importados com sucesso!


## Configuração de API Keys

Configure suas chaves de API nas variáveis de ambiente antes de executar.


In [ ]:
# Configurar API Keys
# IMPORTANTE: Substitua pelos seus valores reais ou use variáveis de ambiente
os.environ["GROQ_API_KEY"] = "sua-groq-api-key-aqui"
os.environ["TAVILY_API_KEY"] = "sua-tavily-api-key-aqui"

print("⚠️ Lembre-se de configurar suas API keys antes de executar!")


## Carregar Dados de Exemplo

Vamos carregar uma amostra do dataset 20 Newsgroups para testar.


In [ ]:
# Carregar uma amostra do dataset
# Se já tiver amostras salvas, pode usar diretamente
sample_file = "data/samples/rec.sport.hockey___1.txt"

# Se não existir, vamos gerar uma amostra
if not os.path.exists(sample_file):
    print("Gerando amostras do dataset 20 Newsgroups...")
    samples = fetch_newsgroups_samples(num_samples=1)
    sample_file = samples[0] if samples else None

if sample_file and os.path.exists(sample_file):
    # Carregar texto
    raw_text = get_text_from_file(sample_file)
    ground_truth = extract_ground_truth_from_filename(sample_file)
    
    print(f"📄 Arquivo: {os.path.basename(sample_file)}")
    print(f"🏷️ Categoria Real: {ground_truth}")
    print(f"\n📝 Texto (primeiros 500 caracteres):\n{raw_text[:500]}...")
else:
    print("❌ Erro ao carregar amostra")


## Pré-processamento do Texto

Limpar e preparar o texto para classificação.


In [ ]:
# Limpar texto
cleaned_text = clean_text(raw_text)
print(f"✅ Texto limpo ({len(cleaned_text)} caracteres)")
print(f"\nPrimeiros 300 caracteres do texto limpo:\n{cleaned_text[:300]}...")


## Criar Agentes

Inicializar os três agentes do sistema VerbaFlow.


In [ ]:
# Criar LLM
llm = get_llm()

# Criar agentes
analyst = create_analyst_agent(llm)
researcher = create_researcher_agent(llm)
editor = create_editor_agent(llm)

print("✅ Agentes criados com sucesso!")
print(f"  - {analyst.role}")
print(f"  - {researcher.role}")
print(f"  - {editor.role}")


## Criar Tasks

Definir as três tasks sequenciais do pipeline.


In [ ]:
# Criar tasks
task1 = create_classification_task(analyst, cleaned_text)
task2 = create_enrichment_task(researcher, task1)
task3 = create_reporting_task(editor, task1, task2)

print("✅ Tasks criadas com sucesso!")
print(f"  - Task 1: Classificação")
print(f"  - Task 2: Enriquecimento (depende de Task 1)")
print(f"  - Task 3: Relatório (depende de Task 1 e 2)")


## Executar Crew

Executar o pipeline completo com os agentes trabalhando em sequência.


In [ ]:
# Definir OPENAI_API_KEY como dummy para evitar erro de importação
# (CrewAI tenta importar OpenAI mesmo quando usamos LLM customizado)
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "dummy-key-to-prevent-openai-import-error"

# Definir OPENAI_API_KEY como dummy para evitar erro de importação
# (CrewAI tenta importar OpenAI mesmo quando usamos LLM customizado)
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "dummy-key-to-prevent-openai-import-error"

# Criar crew com LLM configurado explicitamente com LLM configurado explicitamente
crew = Crew(
    agents=[analyst, researcher, editor],
    tasks=[task1, task2, task3],
    process=Process.sequential,
    verbose=True,
    llm=llm  # Especificar LLM explicitamente
)

print("🚀 Executando VerbaFlow...")
print("=" * 50)

# Executar
result = crew.kickoff()

print("=" * 50)
print("✅ Execução concluída!")


## Resultados

Exibir o resultado final e validar a classificação.


In [ ]:
import re

# Extrair categoria prevista
predicted_category = ""
category_match = re.search(r'Category:\s*([^\n]+)', str(result), re.IGNORECASE)
if category_match:
    predicted_category = category_match.group(1).strip()

# Comparar com ground truth
is_correct = predicted_category.lower() == ground_truth.lower()

print("📊 Resultados da Validação")
print(f"  Categoria Real: {ground_truth}")
print(f"  Categoria Prevista: {predicted_category if predicted_category else 'Não encontrada'}")
print(f"  Status: {'✅ CORRETO' if is_correct else '❌ INCORRETO'}")


## Relatório Completo

Exibir o relatório enriquecido completo gerado pelo Editor Chefe.


In [ ]:
# Exibir relatório completo
print("\n" + "=" * 50)
print("📋 RELATÓRIO ENRIQUECIDO COMPLETO")
print("=" * 50 + "\n")
print(result)
